# 實作3. SQL使用名稱跟位置(url)儲存圖片

> pymysql純存地址參考資料:<br>
>    https://www.796t.com/article.php?id=209513<br>
> 補充資料(JavaScript):<br>
>https://www.zymseo.com/big5/biancheng_310720

> flask儲存圖片的參考網址:
    https://blog.csdn.net/dcrmg/article/details/81987808

## 1-1.在MySQL server建立table來儲存url位置和其他資訊:

In [14]:
import pymysql
import sys
# from sqlalchemy.exc import OperationalError

# 建立連線
conn = pymysql.connect(host="127.0.0.1",
                       port = 3306,
                       user="user1",
                       password="123456",
                       database="team2_test",
                       use_unicode=True)
# 建立遊標
cursor = conn.cursor()

# 提交，不然無法儲存新建或者修改的資料
conn.commit()

# # 建立table的 sql語法
sql = '''CREATE TABLE `team2_test`.`img_address` (
  `img_id` INT NOT NULL,
  `img_name` VARCHAR(50) NULL,
  `img_url` VARCHAR(50) NULL,
  `datetime` DATETIME NULL,
  PRIMARY KEY (`img_id`));'''

# sql = '''DROP TABLE img_address'''

try: 
    cursor.execute(sql)   

except:
    print("[ERROR]: Table 'img_address' already exists")

    
# 關閉遊標
cursor.close()
# 關閉連線
conn.close()
    
print("============")
print("Done! Table was built!")

Done! Table was built!


## 1-2. 圖片讀取存到路徑下:

In [20]:
import cv2
import pymysql
import sys
import os

# os.getcwd()
img = cv2.imread('./practice_img/trello_head.png')
print(img.shape)

(162, 175, 3)


# 2.flask架設

## 2-1.下載模組:

In [24]:
!pip install flask

In [31]:
!pip install opencv-python

## 2-2. 架設flask

### (1)最簡單的網站:

In [4]:
from flask import Flask

app = Flask(__name__,static_url_path = '/resource',static_folder='static')

@app.route('/')
def index():
    return 'Hello Flask'

@app.route('/')


if __name__ == "__main__":
#     app.run()
    app.run(host='0.0.0.0',port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.12:5000/ (Press CTRL+C to quit)
192.168.1.12 - - [17/Nov/2021 08:57:18] "GET / HTTP/1.1" 200 -
192.168.1.12 - - [17/Nov/2021 08:57:18] "GET /favicon.ico HTTP/1.1" 404 -


In [8]:
from flask import Flask

app = Flask(__name__,static_url_path = '/resource',static_folder='./flask/static')

@app.route('/')
def index():
    return 'Hello Flask'

# @app.route('/')


if __name__ == "__main__":
#     app.run()
    app.run(host='0.0.0.0',port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.12:5000/ (Press CTRL+C to quit)
192.168.1.12 - - [17/Nov/2021 09:16:38] "GET / HTTP/1.1" 200 -
192.168.1.12 - - [17/Nov/2021 09:17:33] "GET /resource/images/trello_head.png HTTP/1.1" 200 -


><br>
>

# 3.別人寫的範本: ((((((尚未完成))))))
>參考網站:<br>
    https://blog.csdn.net/dcrmg/article/details/81987808

In [3]:
from flask import Flask, render_template, request, redirect, url_for, make_response, jsonify
from werkzeug.utils import secure_filename
import os
import cv2
import time

from datetime import timedelta

#設置允許的文件格式:
ALLOWED_EXTENSIONS = set(['png','jpg','JPG','PNG','bmp'])

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.',1)[1] in ALLOWED_EXTENSIONS

app = Flask(__name__)

#設置靜態文件緩存過期時間
app.send_file_max_age_default = timedelta(second=1)

# @app.route('/upload', method=['POST','GET'])
@app.route('/upload', method=['POST','GET'])    #添加路徑
def upload():
    if request.method == 'POST':
        f = request.files['files']
        
        if not (f and allowed_file(f.filename)):
            return jsonify